In [29]:
from googleapiclient.discovery import build
from dateutil import parser
import pandas as pd
from IPython.display import JSON
import os

In [30]:
api_key = os.environ.get('api_key')

channel_ids = ['UCJQJAI7IjbLcpsjWdSzYz0Q',
               'UCnz-ZXXER4jOvuED5trXfEA',
               'UCm9COMxXKm05BQWNv-IpyPg',
               'UCWv7vMbMWH4-V0ZXdmDpPBA',
               'UChY0CVDko-10jP2nepUDlzw',
               'UC8w4I8t2OpqoOpzzNT1c2dg',
               'UCgiMlstKdHZgyRcFtErAr2A',
               'UC8butISFwT-Wl7EV0hUK0BQ',
               'UCtYLUTtgS3k1Fg4y5tAhLbw',
               'UCu4RG-8B9zrVvVM5z-p_tJA',
               'UCd5U7ojVEID9_cWoj-AXaRA',
               'UCZID6dKj9-WXyV81nmqdWGA',
               'UC5DNytAJ6_FISueUfzZCVsw',
               'UCifWfwxTfOYYoczCWmIY8bA',
               'UCFbNIlppjAuEX4znoulh0Cw',
               'UCFbNIlppjAuEX4znoulh0Cw',
               'UCmXmlB4-HJytD7wek0Uo97A',
               'UCxmnwO7dNrwF-jO2miZpyww',
               'UCh9nVJoWXmFb7sLApWGcLPQ',
               'UC5zx8Owijmv-bbhAK6Z9apg',
               'UCfzlCWGWYyIQ0aLC5w48gBQ'
              ]

In [31]:
api_service_name = "youtube"
api_version = "v3"

# Get credentials and create an API client
youtube = build(
    api_service_name, api_version, developerKey=api_key)

In [32]:
def get_channel_stats(youtube, channel_ids):
    
    """
    Get channel stats
    
    Params:
    ------
    youtube: build object of Youtube API
    channel_ids: list of channel IDs
    
    Returns:
    ------
    dataframe with all channel stats for each channel ID
    
    """
    
    all_data = []
    
    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(channel_ids)
    )
    response = request.execute()

    # loop through items
    for item in response["items"]:
        data = {'channelTitle': item['snippet']['title'],
                'subscribers': item['statistics']['subscriberCount'],
                'totalViews': item['statistics']['viewCount'],
                'totalVideos': item['statistics']['videoCount']     
        }
        
        all_data.append(data)
        
    return(pd.DataFrame(all_data))

    

In [ ]:
def get_video_details(youtube, video_ids):

    all_video_info = []
    
    for i in range(0, len(video_ids), 50):
        request = youtube.videos().list(
            part="snippet,contentDetails,statistics",
            id=','.join(video_ids[i:i+50])
        )
        response = request.execute() 

        for video in response['items']:
            stats_to_keep = {'snippet': ['channelTitle', 'title', 'description', 'tags', 'publishedAt'],
                             'statistics': ['viewCount', 'likeCount', 'favouriteCount', 'commentCount'],
                             'contentDetails': ['duration', 'definition', 'caption']
                            }
            video_info = {}
            video_info['video_id'] = video['id']

            for k in stats_to_keep.keys():
                for v in stats_to_keep[k]:
                    try:
                        video_info[v] = video[k][v]
                    except:
                        video_info[v] = None

            all_video_info.append(video_info)
    
    return pd.DataFrame(all_video_info)

In [42]:
ch = get_channel_stats(youtube, channel_ids)

ch.head(20)

,channelTitle,subscribers,totalViews,totalVideos
0,Lingala - Institute,1550,98985,88
1,Thu Vu data analytics,47900,1151427,43
2,MODDED WARFARE,312000,47272948,555
3,Techno Phoenix,157,45669,41
4,JavaScript Mastery,387000,18769217,81
5,Programming with Mosh,2690000,129075018,171
6,Tech Tutorial,140000,8634729,161
7,WebbyFan.com,121000,43277123,329
8,BharatiDWConsultancy,29300,5071762,487
9,StatQuest with Josh Starmer,779000,39783284,218


In [34]:
df2 = pd.read_csv(r"C:\Users\wled3\ytdf16.csv")


In [35]:
vid = get_video_details(youtube, ['d56mG7DezGs', '-SejlHOXexc', 'jwrYRirf5lo'])
vid2 = get_video_details(youtube, ['6hzuNwRHhwE'])



In [36]:
vid = pd.merge(vid, ch)

vid2 = pd.merge(vid2, ch)


In [37]:
vid3 = vid.append(vid2)


C:\Users\wled3\AppData\Local\Temp\ipykernel_2124\4124573275.py:1: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  vid3 = vid.append(vid2)


In [38]:
dfs = pd.read_csv(r"C:\Users\wled3\ytscores.csv")

vid_ids = dfs['video_id'].to_list()


In [39]:
df_f = get_video_details(youtube, vid_ids)


In [44]:
df_f = pd.merge(df_f, ch)
df_f = pd.merge(df_f, dfs)

df_f["channelTitle"].nunique()


19